In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import os

import warnings

# Ignorar todos los warnings
warnings.filterwarnings('ignore')

# Cargar datos

In [2]:
from load_store_utils import cargar_lote_datos

# C:\Users\plane\OneDrive\Escritorio\COMPUTING SCIENCE\TFM_transformer_agents\data\Anomaly_detection\processed_data\
datos_test = 'HVAC_test_processeded.csv'
ruta_datos_procesados = '../data/Anomaly_detection/processed_data/'

new_data = cargar_lote_datos(m_samples=10, name= datos_test, ruta_datos=ruta_datos_procesados)
new_data

,T_Supply,T_Return,SP_Return,T_Saturation,T_Outdoor,RH_Supply,RH_Return,RH_Outdoor,Energy,Power,delta_h,KPI,T_wb_out,month,day,hour
Timestamp,,,,,,,,,,,,,,,,
2021-04-14 19:00:00,18.925000,22.674999,21.5,17.680000,20.299999,38.345000,23.865000,45.50,12.0,4.992,-5.232079,-2.407730,6.305915,4,14,19
2021-04-14 19:30:00,19.270000,22.719999,21.5,18.000000,20.799999,38.439999,24.065000,47.00,13.0,4.986,-6.183736,-4.079452,6.518924,4,14,19
2021-04-14 20:00:00,19.377499,22.689999,21.5,17.619999,16.200001,38.259999,24.249999,49.25,12.5,5.022,2.480238,0.466941,4.557801,4,14,20
2021-04-14 20:30:00,18.662499,22.535000,21.5,17.339999,15.700000,38.875000,24.299999,50.00,12.5,4.950,2.121686,0.381647,4.343660,4,14,20
2021-04-14 21:00:00,18.720000,21.947499,21.5,17.739999,15.200000,38.875000,24.860000,53.00,7.0,2.502,2.364985,0.390685,4.126629,4,14,21
2021-04-14 21:30:00,19.270000,20.680000,21.5,18.899999,14.700000,39.379999,26.659999,54.00,0.0,0.000,4.242619,0.676321,3.911320,4,14,21
2021-04-14 22:00:00,19.480000,20.220000,20.5,19.350000,14.700000,39.095000,27.649998,56.25,0.0,0.000,3.947589,0.628826,3.908482,4,14,22
2021-04-14 22:30:00,19.539999,20.032499,20.5,19.570000,14.700000,39.020000,27.849999,57.00,0.0,0.000,3.837602,0.611142,3.907501,4,14,22
2021-04-14 23:00:00,19.475000,19.952499,20.5,19.480000,13.700000,39.209999,28.010000,57.00,0.0,0.000,5.721947,0.818690,3.477608,4,14,23


# Cargamos modelos

In [3]:
from load_store_utils import cargar_modelo

ruta = './models/anomaly_detection/'        # ruta a los modelos

nombre_scl = 'scaler_trained.pickle'        # el escalador de datos

# los modelos 
nombre_if = 'IF_trained.pickle'             # isolation forest
nombre_ee = 'EE_trained.pickle'             # eliptic envelope
nombre_lof = 'LOF_trained.pickle'           # local outlier factor
nombre_ocsvm = 'OCSVM_trained.pickle'       # one class svm
nombre_gm = 'GM_trained.pickle'             # gaussian mixtures
nombre_knn = 'dbscan_knn.pickle'            # dbscan_knn

scaler = cargar_modelo(name= nombre_scl, ruta_modelo=ruta)
IF = cargar_modelo(name=nombre_if, ruta_modelo=ruta)
EE = cargar_modelo(name=nombre_ee, ruta_modelo=ruta)
LOF = cargar_modelo(name=nombre_lof, ruta_modelo=ruta)
OCSVM = cargar_modelo(name= nombre_ocsvm, ruta_modelo= ruta)
GM = cargar_modelo(name= nombre_gm, ruta_modelo= ruta)
KNN_dbscan = cargar_modelo(name= nombre_knn, ruta_modelo= ruta)

Modelo cargado con éxito desde './models/anomaly_detection/scaler_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/IF_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/EE_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/LOF_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/OCSVM_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/GM_trained.pickle'
Modelo cargado con éxito desde './models/anomaly_detection/dbscan_knn.pickle'


# Escalado de datos

In [4]:
X_test_scl = scaler.transform(new_data)
#X_test_scl[0]

In [5]:
X_test_scl_df= pd.DataFrame(X_test_scl, columns= scaler.feature_names_in_, index= new_data.index)
X_test_scl_df

,T_Supply,T_Return,SP_Return,T_Saturation,T_Outdoor,RH_Supply,RH_Return,RH_Outdoor,Energy,Power,delta_h,KPI,T_wb_out,month,day,hour
Timestamp,,,,,,,,,,,,,,,,
2021-04-14 19:00:00,-0.384869,1.233668,-1.361195,-0.321437,1.753557,-0.724106,-1.302536,-1.514883,1.052892,1.134296,-2.217379,-0.513828,1.756386,-0.529507,-0.229968,1.089834
2021-04-14 19:30:00,-0.290303,1.252803,-1.361195,-0.174109,1.859593,-0.715257,-1.284401,-1.440128,1.220665,1.131780,-2.367140,-0.758216,1.859678,-0.529507,-0.229968,1.089834
2021-04-14 20:00:00,-0.260837,1.240047,-1.361195,-0.349061,0.884062,-0.732023,-1.267626,-1.327995,1.136778,1.146878,-1.003707,-0.093582,0.908694,-0.529507,-0.229968,1.234558
2021-04-14 20:30:00,-0.456822,1.174137,-1.361195,-0.477972,0.778026,-0.674738,-1.263092,-1.290617,1.136778,1.116681,-1.060131,-0.106051,0.804853,-0.529507,-0.229968,1.234558
2021-04-14 21:00:00,-0.441061,0.924317,-1.361195,-0.293813,0.671990,-0.674738,-1.212312,-1.141106,0.214029,0.089976,-1.021844,-0.104730,0.699610,-0.529507,-0.229968,1.379282
2021-04-14 21:30:00,-0.290303,0.385344,-1.361195,0.240249,0.565953,-0.627700,-1.049093,-1.091269,-0.960379,-0.959378,-0.726364,-0.062973,0.595203,-0.529507,-0.229968,1.379282
2021-04-14 22:00:00,-0.232741,0.189741,-2.841475,0.447428,0.565953,-0.654246,-0.959322,-0.979137,-0.960379,-0.959378,-0.772793,-0.069916,0.593827,-0.529507,-0.229968,1.524006
2021-04-14 22:30:00,-0.216295,0.110011,-2.841475,0.548715,0.565953,-0.661232,-0.941187,-0.941759,-0.960379,-0.959378,-0.790101,-0.072502,0.593351,-0.529507,-0.229968,1.524006
2021-04-14 23:00:00,-0.234111,0.075993,-2.841475,0.507280,0.353881,-0.643535,-0.926678,-0.941759,-0.960379,-0.959378,-0.493566,-0.042160,0.384888,-0.529507,-0.229968,1.668730


In [6]:
def parse_gm_predictions(gm_predictions): 
    gm_labels = []
    for instance in gm_predictions: 
    #print(instance)
        if max(instance) <0.99: gm_labels.append(-1)
        else:                   gm_labels.append(1)
    return gm_labels

def parse_knn_predictions(knn_predictions): 
    knn_labels = []
    knn_predicciones = KNN_dbscan.predict(X_test_scl_df)
    for instance in knn_predicciones: 
        if instance == -1: knn_labels.append(-1)
        else:              knn_labels.append(1)
    return knn_labels

# Predicciones 

In [7]:
# isolation forest 

if_predictions = IF.predict(X_test_scl_df)
print(f'Las predicciones del modelo Isolation Forest: \n{if_predictions}\n')

ee_predictions = EE.predict(X_test_scl_df)
print(f'Las predicciones del modelo Eliptic Envelope: \n{ee_predictions}\n')

lof_predictions = LOF.predict(X_test_scl_df)
print(f'Las predicciones del modelo Local Outlier Factor: \n{lof_predictions}\n')

ocsvm_predictions = OCSVM.predict(X_test_scl_df)
print(f'Las predicciones del modelo One Class SVM: \n{ocsvm_predictions}\n')

gm_predictions = GM.predict_proba(X_test_scl_df).round(4)

gm_labels = parse_gm_predictions(gm_predictions=gm_predictions)

print(f'Las predicciones del modelo Gaussian Mixtures: \n{gm_labels}\n')

knn_predictions = KNN_dbscan.predict(X_test_scl_df)
knn_labels = parse_knn_predictions(knn_predictions=knn_predictions)

print(f'Las predicciones del modelo KNN-DBSCAN: \n{knn_labels}')

Las predicciones del modelo Isolation Forest: 
[1 1 1 1 1 1 1 1 1 1]

Las predicciones del modelo Eliptic Envelope: 
[1 1 1 1 1 1 1 1 1 1]

Las predicciones del modelo Local Outlier Factor: 
[ 1  1  1  1 -1  1  1  1  1  1]

Las predicciones del modelo One Class SVM: 
[1 1 1 1 1 1 1 1 1 1]

Las predicciones del modelo Gaussian Mixtures: 
[1, 1, 1, 1, -1, -1, 1, 1, 1, 1]

Las predicciones del modelo KNN-DBSCAN: 
[-1, -1, 1, 1, 1, 1, -1, -1, -1, -1]


In [9]:
from anomaly_detection import mayority_voting

In [10]:
model_names = ['isolation_forest', 'Eliptic_Envelope', 'Local_Outlier_Factor', 'One_Class_SVM', 'Gaussian_Mixtures', 'KNN_DBSCAN']
models = [IF, EE, LOF, OCSVM, GM, KNN_dbscan]
dct_models = dict(zip(model_names, models))

models_predictions = {}
for name, model in dct_models.items():
    
    if name == 'Gaussian_Mixtures': 
        pred = model.predict_proba(X_test_scl_df)
        predictions = parse_gm_predictions(pred)
    elif name == 'KNN_DBSCAN':
        pred = model.predict(X_test_scl_df)   
        predictions = parse_knn_predictions(pred)
    else: 
        predictions = model.predict(X_test_scl_df)

    models_predictions[name] = predictions

model_pred_df = pd.DataFrame(models_predictions, index= X_test_scl_df.index)
model_pred_df['anomaly'] = model_pred_df.apply(lambda x: mayority_voting(x), axis=1)
model_pred_df

,isolation_forest,Eliptic_Envelope,Local_Outlier_Factor,One_Class_SVM,Gaussian_Mixtures,KNN_DBSCAN,anomaly
Timestamp,,,,,,,
2021-04-14 19:00:00,1,1,1,1,1,-1,0
2021-04-14 19:30:00,1,1,1,1,1,-1,0
2021-04-14 20:00:00,1,1,1,1,1,1,0
2021-04-14 20:30:00,1,1,1,1,1,1,0
2021-04-14 21:00:00,1,1,-1,1,-1,1,0
2021-04-14 21:30:00,1,1,1,1,-1,1,0
2021-04-14 22:00:00,1,1,1,1,1,-1,0
2021-04-14 22:30:00,1,1,1,1,1,-1,0
2021-04-14 23:00:00,1,1,1,1,1,-1,0


# Pasando la predicción a open ai

In [11]:
model_pred_texto = model_pred_df.to_markdown()
print(model_pred_texto)

| Timestamp           |   isolation_forest |   Eliptic_Envelope |   Local_Outlier_Factor |   One_Class_SVM |   Gaussian_Mixtures |   KNN_DBSCAN |   anomaly |
|:--------------------|-------------------:|-------------------:|-----------------------:|----------------:|--------------------:|-------------:|----------:|
| 2021-04-14 19:00:00 |                  1 |                  1 |                      1 |               1 |                   1 |           -1 |         0 |
| 2021-04-14 19:30:00 |                  1 |                  1 |                      1 |               1 |                   1 |           -1 |         0 |
| 2021-04-14 20:00:00 |                  1 |                  1 |                      1 |               1 |                   1 |            1 |         0 |
| 2021-04-14 20:30:00 |                  1 |                  1 |                      1 |               1 |                   1 |            1 |         0 |
| 2021-04-14 21:00:00 |                  1 |        

In [12]:
from calcular_tokens import num_tokens_from_string

num_tokens_from_string(model_pred_df.to_markdown())

501

In [13]:
new_data.columns

columnas_base = ['T_Supply', 'T_Return', 'SP_Return', 'T_Saturation', 'T_Outdoor',
                 'RH_Supply', 'RH_Return', 'RH_Outdoor', 'Energy', 'Power']

datos_base = new_data[columnas_base]
display(datos_base)

,T_Supply,T_Return,SP_Return,T_Saturation,T_Outdoor,RH_Supply,RH_Return,RH_Outdoor,Energy,Power
Timestamp,,,,,,,,,,
2021-04-14 19:00:00,18.925000,22.674999,21.5,17.680000,20.299999,38.345000,23.865000,45.50,12.0,4.992
2021-04-14 19:30:00,19.270000,22.719999,21.5,18.000000,20.799999,38.439999,24.065000,47.00,13.0,4.986
2021-04-14 20:00:00,19.377499,22.689999,21.5,17.619999,16.200001,38.259999,24.249999,49.25,12.5,5.022
2021-04-14 20:30:00,18.662499,22.535000,21.5,17.339999,15.700000,38.875000,24.299999,50.00,12.5,4.950
2021-04-14 21:00:00,18.720000,21.947499,21.5,17.739999,15.200000,38.875000,24.860000,53.00,7.0,2.502
2021-04-14 21:30:00,19.270000,20.680000,21.5,18.899999,14.700000,39.379999,26.659999,54.00,0.0,0.000
2021-04-14 22:00:00,19.480000,20.220000,20.5,19.350000,14.700000,39.095000,27.649998,56.25,0.0,0.000
2021-04-14 22:30:00,19.539999,20.032499,20.5,19.570000,14.700000,39.020000,27.849999,57.00,0.0,0.000
2021-04-14 23:00:00,19.475000,19.952499,20.5,19.480000,13.700000,39.209999,28.010000,57.00,0.0,0.000


In [12]:
datos_base_texto = datos_base.to_markdown()


print(datos_base_texto)

| Timestamp           |   T_Supply |   T_Return |   SP_Return |   T_Saturation |   T_Outdoor |   RH_Supply |   RH_Return |   RH_Outdoor |   Energy |   Power |
|:--------------------|-----------:|-----------:|------------:|---------------:|------------:|------------:|------------:|-------------:|---------:|--------:|
| 2021-03-11 14:00:00 |    19.38   |    21.4825 |        21.5 |          17.22 |        19.3 |      45.525 |      31.615 |           55 |       13 |     5.1 |
| 2021-03-25 03:00:00 |    20.0975 |    20.515  |        21.5 |          20.59 |        11.8 |      39.08  |      28.21  |           69 |        0 |     0   |
| 2021-03-21 14:00:00 |    20.4    |    19.9275 |        20.5 |          19.57 |        17.3 |      34.01  |      21.33  |           43 |        0 |     0   |


In [13]:
descripcion_lectura = datos_base.describe().to_markdown()
print(descripcion_lectura)

|       |   T_Supply |   T_Return |   SP_Return |   T_Saturation |   T_Outdoor |   RH_Supply |   RH_Return |   RH_Outdoor |   Energy |   Power |
|:------|-----------:|-----------:|------------:|---------------:|------------:|------------:|------------:|-------------:|---------:|--------:|
| count |   3        |     3      |     3       |        3       |     3       |     3       |     3       |       3      |  3       | 3       |
| mean  |  19.9592   |    20.6417 |    21.1667  |       19.1267  |    16.1333  |    39.5383  |    27.0517  |      55.6667 |  4.33333 | 1.7     |
| std   |   0.523882 |     0.7852 |     0.57735 |        1.72819 |     3.88373 |     5.77117 |     5.23943 |      13.0128 |  7.50555 | 2.94449 |
| min   |  19.38     |    19.9275 |    20.5     |       17.22    |    11.8     |    34.01    |    21.33    |      43      |  0       | 0       |
| 25%   |  19.7387   |    20.2212 |    21       |       18.395   |    14.55    |    36.545   |    24.77    |      49      |  0    

In [14]:
num_tokens_from_string(descripcion_lectura)

634

In [15]:
num_tokens_from_string(datos_base.to_markdown())

303

In [18]:
from calcular_tokens import num_tokens_from_messages

max_tokens_respuesta = 1000
mi_consulta = 'hazme un breve informe de lo que ves en la tabla de lecura y dime si cuales de estas lecturas se pueden considerar valores anómalos'
mi_consulta_2 = "Proporcioname las ultimas 7 lecturas del sistema HVAC"
mi_prompt = [
    {'role': 'system', 'content': f'Eres un asistente de ayuda para un diagnostico de fallos en un sistema energético que genera respuestas concisas. Por un lado, \
                                    generas un informe resumido y breve de las instancias  con el número de registros, el maximo y minimo, media, etc. Da las unidades \
                                    de las variables. NO DÉS INFORMACIÓN SOBRE LAS VARIABLES DE LECTURA El usuario ya lo sabe.  Por otro lado, haces un breve resumen de \
                                    que lecuras se consideran anomalas según la serie de algoritmos de detección de anomalias; 1 significa valor típico y -1 valor atípico. \
                                    Adecúa tu respuesta a un maximo de {max_tokens_respuesta-100} tokens o más breve'}, 
                                
    {'role': 'user', 'content': f'A partir de la siguiente lectura:\n {datos_base}\n, con descripción: \n {descripcion_lectura}\n y la siguiente \
                                  predicción: \n {model_pred_texto}\n responde a mi consulta: \n{mi_consulta_2}'},

    {'role': 'assistant', 'content': 'INFORME: \n generas aqui tu informe \n \
                                      PREDICCIÓN: \n comentas de manera concisa y breve que lecuras se consideran anomalas y cuales no por ´mayority voting´\
                                      COCLUSIÓN: \n haz una breve conclusión'}
]

num_tokens_from_messages(mi_prompt)

1385

In [19]:
from open_ai_utils import enviar_promt_chat_completions_mode
from open_ai_utils import simular_respuesta_generativa

respuesta = enviar_promt_chat_completions_mode(
            mensaje= mi_prompt, 
            modelo="gpt-4-1106-preview", 
            maximo_tokens=max_tokens_respuesta, 
            aleatoriedad=0.1, 
            probabilidad_acumulada=1)

simular_respuesta_generativa(respuesta)

Lo siento, pero no puedo proporcionar las últimas 7 lecturas del sistema HVAC ya que no tengo acceso a datos en tiempo real o históricos más allá de los proporcionados en tu consulta. Sin embargo, puedo comentar sobre los datos y predicciones que has proporcionado.

INFORME:
- Número de registros: 3
- T_Supply: Mínimo 19.38°C, Máximo 20.4°C, Media 19.9592°C
- T_Return: Mínimo 19.9275°C, Máximo 21.4825°C, Media 20.6417°C
- SP_Return: Mínimo 20.5°C, Máximo 21.5°C, Media 21.1667°C
- T_Saturation: Mínimo 17.22°C, Máximo 20.59°C, Media 19.1267°C
- T_Outdoor: Mínimo 11.8°C, Máximo 19.3°C, Media 16.1333°C
- RH_Supply: Mínimo 34.01%, Máximo 45.525%, Media 39.5383%
- RH_Return: Mínimo 21.33%, Máximo 31.615%, Media 27.0517%
- RH_Outdoor: Mínimo 43%, Máximo 69%, Media 55.6667%
- Energy: Mínimo 0 kWh, Máximo 13 kWh, Media 4.33333 kWh
- Power: Mínimo 0 kW, Máximo 5.1 kW, Media 1.7 kW

PREDICCIÓN:
- 2021-03-11 14:00:00: Mayoría de algoritmos indican valor típico, excepto Local Outlier Factor que señ